# Generating Jacccard distance matrices efficiently

## Setup

In [1]:
import sys

import scipy
import umap
import pandas as pd

from scipy.spatial import distance as scd
from sklearn import metrics as skm
from matplotlib import pyplot as plt

In [7]:
sys.path.append("/usr/local/dev_scripts")

import similarity_utils as su

SyntaxError: invalid syntax (input_parser.py, line 11)

## Parse input

In [ ]:
main_df = su.input_parser.read_input("/home/rstudio/local_files/example_data/pseudo_tabular_format.csv")
main_df.head()

## Calculate distances

In [ ]:
%time loop_distances = calculate_jaccard(main_df, "loop")
loop_distances.head()

In [ ]:
%time pandas_distances = calculate_jaccard(main_df, "pandas")
pandas_distances.head()

In [ ]:
%time scikit_distances = calculate_jaccard(main_df)
scikit_distances.head()

## Visualize results

In [ ]:
U = umap.UMAP(metric='precomputed')
umap_coords = U.fit_transform(scikit_distances)